# Notation

For a linear elastic material model, the ufl form `a` for the stiffness matrix can be created like this:

In [104]:
import dolfin as df
import numpy as np

E = 1.0
nu = 0.3
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / (2.0 * (1 + nu))

mesh = df.UnitCubeMesh(1, 1, 1)

V = df.VectorFunctionSpace(mesh, "Lagrange", 2)

u = df.TestFunction(V)
v = df.TrialFunction(V)


def eps(u):
    return df.sym(df.grad(u))


def sigma(u):
    return lmbda * df.tr(eps(u)) * df.Identity(3) + 2.0 * mu * eps(u)


a = df.inner(eps(v), sigma(u)) * df.dx

The last line represents the integral
$$
a = \int \sigma : \varepsilon \mathrm d x
$$
which can be expressed with the elastic stiffness tensor $D_e$ for the case of linear elasticity:
$$
a = \int   \varepsilon :(D_e :\varepsilon) \mathrm d x.
$$

For a nonlinear constitutive model, the elasticity tensor must be replaced with a consistent tangent, which is generally not constant on the mesh.

## Mandel notation

It is useful to write the symmetric tensors as matrices and vectors, such that $D_e \in \mathbb R ^{4,4}$, $\sigma, \varepsilon \in \mathbb R ^4$ for plane strain or $D_e \in \mathbb R ^{6,6}$, $\sigma, \varepsilon \in \mathbb R ^6$ for 3d strains and stresses.
For our purposes, the [Mandel notation](https://en.wikipedia.org/wiki/Voigt_notation#Mandel_notation) is most useful. 

$$
a = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{12} & a_{22} & a_{23}\\
a_{13} & a_{23} & a_{33}
\end{bmatrix}
\hat= \left[ a_{11},a_{22},a_{33},\sqrt 2 a_{23},\sqrt 2 a_{13},\sqrt 2 a_{12}\right]^\top = \vec a
$$

It has the advantage that an inner product of two symmetric tensors can be expressed by the inner product of to vectors. The same is true for outer products.

$$
a:b= a_{11}b_{11}+a_{22}b_{22} + a_{33}b_{33} + 2a_{23}b_{23}+2a_{13}b_{13}+2a_{12}b_{12}=\vec a \cdot \vec b
$$

Therefore:

$$
a = \int   \vec\varepsilon \cdot D_e \cdot\vec\varepsilon \mathrm d x
$$

The computation of the strain is changed to

In [116]:
def eps_mandel(u):
    e = df.sym(df.grad(u))
    return df.as_vector(
        [
            e[0, 0],
            e[1, 1],
            e[2, 2],
            2 ** 0.5 * e[1, 2],
            2 ** 0.5 * e[0, 2],
            2 ** 0.5 * e[0, 1],
        ]
    )

## Quadrature spaces

In order to evaluate the integral, the integrand needs to be evaluated at the quadrature points. For a nonlinear material model, the tangent is generally not the constant elastic tangent. Using a QuadratureSpace of dimension $6\times6$, we can store one tangent for each integration point and use it in a ufl expression. 

In [117]:
import warnings
from ffc.quadrature.deprecation import QuadratureRepresentationDeprecationWarning

df.parameters["form_compiler"]["representation"] = "quadrature"
warnings.simplefilter("ignore", QuadratureRepresentationDeprecationWarning)

metadata = {"quadrature_degree": 2, "quadrature_scheme": "default"}
dxm = df.dx(metadata=metadata)

WCe = df.TensorElement(
    "Quadrature", mesh.ufl_cell(), degree=2, shape=(6, 6), quad_scheme="default",
)
WC = df.FunctionSpace(mesh, WCe)
C = df.Function(WC)

We will now fill the empty space `C` repeatedly with the elasticity matrix, by directly accessing the vector which stores the numerical values of the Function object. For a more complex model, this would mean iterating over all integration points and saving the flattened tangent at the right point of the vector.

In [118]:
D = np.array(
    [
        [2 * mu + lmbda, lmbda, lmbda, 0, 0, 0],
        [lmbda, 2 * mu + lmbda, lmbda, 0, 0, 0],
        [lmbda, lmbda, 2 * mu + lmbda, 0, 0, 0],
        [0, 0, 0, 2 * mu, 0, 0],
        [0, 0, 0, 0, 2 * mu, 0],
        [0, 0, 0, 0, 0, 2 * mu],
    ]
)

n = WC.dim() // (6 * 6)

C.vector().set_local(np.tile(D.flatten(), n))
C.vector().apply("insert")

Assembling the stiffnes matrices from the two approaches and calculating the difference shows us that both mehtods yield the same results.

In [119]:
a_mandel = df.inner(eps_mandel(u), df.dot(C, eps_mandel(v))) * dxm

K = df.assemble(a)
K_mandel = df.assemble(a_mandel)

print("Difference between matrices:", np.linalg.norm(K.array() - K_mandel.array()))

Difference between matrices: 2.475866723157273e-15
